In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
import numpy as np
from transformers import DistilBertTokenizer, TFDistilBertModel, AutoTokenizer, TrainingArguments, Trainer, DistilBertModel
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch.nn as nn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
path = "sample_data/toyset.csv"
df = pd.read_csv(path, dtype="string")
df['Definition'] = df['Definition'].astype(str)
df = df[['Word', 'Definition']]
df

,Word,Definition
0,Geographical,"""Of or pertaining to geography."""
1,Inextricableness,"""The state of being inextricable."""
2,Papuars,"""The native black race of Papua or New Guinea ..."
3,dark-coated,covered with dark hair
4,Cesura,"""See Caesura."""
...,...,...
233,olive,a tree of some other species of olea or of som...
234,olive,evergreen tree cultivated in the mediterranean...
235,olive,an evergreen tree olea europaea cultivated sin...
236,olive,the tree has been cultivated for its fruit for...


In [ ]:
'''Convert classes to numbers'''
word_dict = {} 
i = 0
for w in df['Word'].unique():
    word_dict[w] = i
    i += 1

'''Convert numbers back to words'''
idx2word = {v:k for k,v in word_dict.items()}

In [ ]:
df_train, df_test = train_test_split(df[['Definition','Word']], test_size=0.2)
df_test, df_val = train_test_split(df_test[['Definition','Word']], test_size=0.5)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

train_enc = tokenizer(df_train['Definition'].to_list(), padding=True, truncation=True, max_length=128)
test_enc = tokenizer(df_test['Definition'].to_list(), padding=True, truncation=True, max_length=128)
val_enc = tokenizer(df_val['Definition'].to_list(), padding=True, truncation=True, max_length=128)

In [ ]:
class RevDictDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        self.labels = self.labels.to_list()
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [ ]:
'''One hot encoding of classes'''
train_label_enum = {k:j+1 for j, k in enumerate(df_train['Word'].unique())}
train_label_enum["<unk>"] = 0
train_num_labels = len(train_label_enum)
idx2token = {idx: token for token, idx in train_label_enum.items()}
df_train['labels'] = df_train['Word'].apply(lambda x: [1.0 if train_label_enum[x]==i else 0.0 for i in range(train_num_labels)])

labels = []
for word in list(df_val['Word']):
  if word in train_label_enum:
    labels.append(train_label_enum[word])
  else:
    labels.append(train_label_enum["<unk>"])
df_val['labels'] = labels

In [ ]:
train_dataset = RevDictDataset(train_enc, df_train['labels'])
val_dataset = RevDictDataset(val_enc, df_val['labels'])

In [ ]:
class BLmodel(nn.Module):
  def __init__(self):
    super(BLmodel, self).__init__()
    self.bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
    # Define LSTM layers
    self.lstm_layer_1 = nn.LSTM(input_size=768, hidden_size=256, num_layers=1, batch_first=True, bidirectional=False)
    self.lstm_layer_2 = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)
    self.lstm_layer_3 = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, batch_first=True, bidirectional=False)
    self.lstm_layer_4 = nn.LSTM(input_size=64, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False)

    # Define output layer
    self.output_layer = nn.Linear(32, len(train_label_enum))

    # Define softmax activation
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_ids, attention_mask=None):
    # Get BERT embeddings
    outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
    bert_embedding = outputs[0]
    # Get LSTM outputs
    lstm_output_1, _ = self.lstm_layer_1(bert_embedding)
    lstm_output_2, _ = self.lstm_layer_2(lstm_output_1)
    lstm_output_3, _ = self.lstm_layer_3(lstm_output_2)
    lstm_output_4, _ = self.lstm_layer_4(lstm_output_3)

    # Get output and apply softmax
    output = self.output_layer(lstm_output_4[:, -1, :])
    output = self.softmax(output)
    return output

In [ ]:
# Instantiate the DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define the training function
def train(model, dataloader, optimizer, num_epochs):
  model.train()
  for epoch in range(num_epochs):
    for batch in dataloader:
      input_ids = batch['input_ids']
      labels = batch['labels']
      optimizer.zero_grad()
      outputs = model(input_ids)
      loss = nn.CrossEntropyLoss()(outputs.view(-1, len(train_label_enum)), labels)
      loss.backward()
      optimizer.step()
    print(f"Epoch {epoch+1} complete. Loss: {loss.item()}")

# Define the training parameters
model = BLmodel()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.05)
num_epochs = 5
train(model, train_loader, optimizer, 5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: ignored

In [ ]:
def evaluate(model, loader):
  model.eval()
  val_predictions = []
  for batch in loader:
      input_ids = batch['input_ids']
      labels = batch['labels']
      outputs = model(input_ids)
      # print(outputs)
      _, predicted = torch.max(outputs, dim=1)
      val_predictions.extend(predicted.flatten().tolist())
  return val_predictions

val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
val_predictions = evaluate(model, val_loader)


In [ ]:
df_val['predicted'] = [idx2token[idx] for idx in val_predictions]
df_val

,Definition,Word,labels,predicted
4,"""See Caesura.""",Cesura,0,olive
92,"""A dance in a circle.""",Roundelay,5,olive
51,a piece of betel nut,betel,23,olive
194,any of numerous shrubs or herbs or vines of th...,nightshade,4,olive
2,"""The native black race of Papua or New Guinea ...",Papuars,0,olive
189,any of numerous shrubs or herbs or vines of th...,nightshade,4,olive
158,to be or become angry an insult that really ma...,burn,14,olive
182,"""Same as Amnion.""",Amnios,0,olive
106,"""To strike a gentle blow.""",Tap,1,olive
32,also called looking glass,mirroring,28,olive
